## Interact with NGSI-LD context broker usign the data model

Uses orion-ld broker

copy updated context to kubernetes using:

In [26]:
from ngsildclient import Entity, Client

In [12]:
host = "localhost"
client = Client(hostname=host, port=9090, port_temporal=9090)
#ctx = "http://nginx:8000/context-ngsi.jsonld"
ctx = "https://uri.etsi.org/ngsi-ld/v1/ngsi-ld-core-context.jsonld"

Connected to Context Broker at localhost:9090 | tenant=N/A | vendor=Scorpio | version=1.1.0-SNAPSHOT

In [13]:
# Create a cow
import datetime
import numpy as np
start = datetime.datetime.fromisoformat("2023-01-01T00:00:00")
cow = Entity("AnimalDailyObserved", "cow4", ctx = ctx)
cow.prop("methaneyield", np.random.randint(30, 60), observedat=start)
cow.prop("net_energy_requirement", np.random.randint(700, 1000), observedat=start)
#cow.tprop("observedAt", start) # This gives errors from orion
cow["observedAt"] = start.isoformat() + "Z" # This works
#cow.obs() This one works for some reason
cow.pprint()
#client.create(cow)

{
  "id": "urn:ngsi-ld:AnimalDailyObserved:cow4",
  "type": "AnimalDailyObserved",
  "@context": "https://uri.etsi.org/ngsi-ld/v1/ngsi-ld-core-context.jsonld",
  "methaneyield": {
    "type": "Property",
    "value": 35,
    "observedAt": "2023-01-01T00:00:00Z"
  },
  "net_energy_requirement": {
    "type": "Property",
    "value": 844,
    "observedAt": "2023-01-01T00:00:00Z"
  },
  "observedAt": "2023-01-01T00:00:00Z"
}

In [22]:
%%time
# Create and update the values
for i in range(10):
    cow = Entity("AnimalDailyObserved", f"ttcow{i}", ctx = ctx)
    cow.prop("methaneyield", np.random.randint(30, 60))
    cow.prop("net_energy_requirement", np.random.randint(700, 1000))
    #cow.tprop("observedAt", start) # This gives errors from orion
    cow["observedAt"] = start.isoformat() + "Z" # This works
    #cow.obs() This one works for some reason
    #cow.pprint()
    client.create(cow)
    for t in range(5):
        tn = start + datetime.timedelta(days=t)
        cow.prop("methaneyield", np.random.randint(30, 60), observedat=tn)
        cow.prop("net_energy_requirement", np.random.randint(700, 1000), observedat=tn)
        #cow.prop("observedAt", start.isoformat())
        #cow["observedAt"] =tn.isoformat() + "Z"
        #cow.pprint()
        client.update(cow)

CPU times: user 309 ms, sys: 0 ns, total: 309 ms
Wall time: 929 ms


In [25]:
animals = client.query("AnimalDailyObserved", ctx=ctx)
client.temporal.get(animals[0])

{'id': 'urn:ngsi-ld:AnimalDailyObserved:ttcow4',
 'type': 'AnimalDailyObserved',
 'methaneyield': [59,
  '2023-01-05T00:00:00Z',
  44,
  '2023-01-02T00:00:00Z',
  46,
  '2023-01-03T00:00:00Z',
  33,
  34,
  '2023-01-01T00:00:00Z',
  54,
  '2023-01-04T00:00:00Z'],
 'net_energy_requirement': [878,
  741,
  '2023-01-01T00:00:00Z',
  887,
  '2023-01-02T00:00:00Z',
  821,
  '2023-01-03T00:00:00Z',
  939,
  '2023-01-04T00:00:00Z',
  770,
  '2023-01-05T00:00:00Z'],
 'ngsi-ld:observedAt': [],
 '@context': ['https://uri.etsi.org/ngsi-ld/v1/ngsi-ld-core-context-v1.3.jsonld']}

## Query time series

In [46]:
#client.temporal.get(animals[0])

In [30]:
# See https://ngsi-ld-tutorials.readthedocs.io/en/latest/short-term-history.html

query using requests:

In [23]:
import requests
import datetime

url = 'http://localhost:9090/ngsi-ld/v1/temporal/entities/' 
headers = {
    'Link': f'<{ctx}>; rel="http://www.w3.org/ns/json-ld#context"; type="application/ld+json"',
}
params = {
    'type': 'AnimalDailyObserved',
    'lastN': 100,
    'timeproperty': 'observedAt',
    #'timeproperty': 'dateObserved',
    'timerel': 'before',
    'timeAt': f'{datetime.datetime.now().isoformat()}',
}

response = requests.get(url, headers=headers, params=params)
#cowdata = response.json()
#cowdata[1]
response.json()

[{'id': 'urn:ngsi-ld:AnimalDailyObserved:ttcow0',
  'type': 'AnimalDailyObserved',
  'methaneyield': [{'type': 'Property',
    'value': 44,
    'instanceId': 'urn:ngsi-ld:95a63141-0190-4f3a-b351-2fe760af0e70',
    'observedAt': '2023-01-05T00:00:00Z'},
   {'type': 'Property',
    'value': 39,
    'instanceId': 'urn:ngsi-ld:e81e473f-5a4b-466c-9ba2-af045cafcef6',
    'observedAt': '2023-01-04T00:00:00Z'},
   {'type': 'Property',
    'value': 49,
    'instanceId': 'urn:ngsi-ld:0f2fb17f-ebe4-4e52-94a2-2b6ec705ef9c',
    'observedAt': '2023-01-03T00:00:00Z'},
   {'type': 'Property',
    'value': 40,
    'instanceId': 'urn:ngsi-ld:f54e044c-9215-4839-b24d-5812ce28623f',
    'observedAt': '2023-01-02T00:00:00Z'},
   {'type': 'Property',
    'value': 58,
    'instanceId': 'urn:ngsi-ld:7ba53662-8f95-4d1b-8ece-50d8be8c3f01',
    'observedAt': '2023-01-01T00:00:00Z'}],
  'net_energy_requirement': [{'type': 'Property',
    'value': 844,
    'instanceId': 'urn:ngsi-ld:4a668f4a-f64d-43dd-a429-1bfa84b

In [20]:
%%timeit
response = requests.get(url, headers=headers, params=params)

5.46 ms ± 426 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
#while True:
#    response = requests.get(url, headers=headers, params=params)